In [65]:
  # import libraries
import keras
import cv2
import os
import tensorflow as tf
from keras.models import Sequential
from glob import glob
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, Conv3D, BatchNormalization
from keras.layers import LeakyReLU

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [66]:
def histogram_equalization(image):
    # Chuyển đổi ảnh sang ảnh xám
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Tính toán histogram của ảnh xám
    hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
    
    # Tính toán histogram tích lũy
    cum_hist = hist.cumsum()
    
    # Chuẩn hóa histogram tích lũy
    cum_hist_normalized = cum_hist / cum_hist.max()
    
    # Ánh xạ lại các giá trị cường độ từ histogram tích lũy chuẩn hóa
    mapping = (cum_hist_normalized * 255).astype(np.uint8)
    
    # Áp dụng phép ánh xạ lại lên ảnh gốc
    equalize_image = mapping[gray_image.astype(np.uint8)]
    
    return equalize_image

# Mount Google Drive
drive.mount('/content/drive')

# Đường dẫn đến thư mục chứa dataset trên Google Drive
dataset_path = '/content/drive/MyDrive/MV/MV FINAL/TRAIN'

# Danh sách nhãn
labels = ['COVID19', 'HEALTHY', 'PNEUMONIA']

processed_images = []
processed_labels = []

# Duyệt qua từng nhãn
for label in labels:
    label_path = dataset_path + '/' + label  # Đường dẫn đến thư mục của nhãn hiện tại
    
    # Duyệt qua từng ảnh trong nhãn
    for image_name in os.listdir(label_path):
        image_path = label_path + '/' + image_name  # Đường dẫn đến ảnh hiện tại

        # Đọc ảnh và áp dụng histogram equalization
        image = cv2.imread(image_path)
        equalized_image = histogram_equalization(image)

        processed_images.append(equalized_image)
        processed_labels.append(label)

# Chuyển đổi danh sách ảnh và nhãn thành mảng numpy
processed_images = np.array(processed_images)
processed_labels = np.array(processed_labels)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-66-d0d54aa2b188>:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  processed_images = np.array(processed_images)


In [67]:
# Kiểm tra kích thước bộ dataset mới
print("Kích thước bộ dataset mới:", processed_images.shape)
print("Số nhãn trong bộ dataset mới:", len(np.unique(processed_labels)))

Kích thước bộ dataset mới: (5478,)
Số nhãn trong bộ dataset mới: 3


In [68]:
label_indices = {}

for idx, label in enumerate(labels):
    label_indices[label] = np.where(processed_labels == label)[0]

# In thông tin về từ điển
for label, indices in label_indices.items():
    print(f"{label}: {len(indices)} ảnh")

COVID19: 140 ảnh
HEALTHY: 1815 ảnh
PNEUMONIA: 3523 ảnh


In [69]:
# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(processed_images, processed_labels, test_size=0.2, random_state=42)

from skimage.transform import resize

#Resize ảnh về cùng 1 kích thước
resized_images_train = []
for image in X_train:
    resized_image = resize(image, (150, 150, 3))
    resized_images_train.append(resized_image)

X_train = np.array(resized_images_train)

resized_images_test = []
for image in X_test:
    resized_image = resize(image, (150, 150, 3))
    resized_images_test.append(resized_image)

X_test = np.array(resized_images_test)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

label_mapping = {'COVID19': 0, 'HEALTHY': 1, 'PNEUMONIA': 2}

# Chuyển đổi nhãn thành dạng số nguyên
y_train = np.array([label_mapping[label] for label in y_train])
y_test = np.array([label_mapping[label] for label in y_test])

# Chuyển đổi nhãn thành one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)


In [70]:
#Define và compile CNN model
model=Sequential()

# Nhân tích chập 32 lần
model.add(Conv2D(32,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same',input_shape=(150,150,3))) 
#Hàm loại bỏ
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(2,2))
# Nhân tích chập 64 lần 
model.add(Conv2D(64,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
#Hàm loại bỏ
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(2,2))
# Nhân tích chập 128 lần 
model.add(Conv2D(128,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
#Hàm loại bỏ
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(2,2))

model.add(Flatten())

model.add(Dense(256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 150, 150, 32)      896       
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 150, 150, 32)      0         
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 75, 75, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 75, 75, 64)        18496     
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 75, 75, 64)        0         
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 37, 37, 64)       0         
 g2D)                                                 

In [71]:
# Biên dịch mô hình
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

#Lưu lại mô hình
model.save("/content/drive/MyDrive/MV/MV FINAL/mv_final.h5")

Epoch 1/20
137/137 [==============================] - 292s 2s/step - loss: 1.2839 - accuracy: 0.6853 - val_loss: 0.6092 - val_accuracy: 0.7290
Epoch 2/20
137/137 [==============================] - 343s 3s/step - loss: 0.5605 - accuracy: 0.7494 - val_loss: 0.5621 - val_accuracy: 0.7336
Epoch 3/20
137/137 [==============================] - 299s 2s/step - loss: 0.5158 - accuracy: 0.7741 - val_loss: 0.5416 - val_accuracy: 0.7655
Epoch 4/20
137/137 [==============================] - 295s 2s/step - loss: 0.4787 - accuracy: 0.7793 - val_loss: 0.5629 - val_accuracy: 0.7372
Epoch 5/20
137/137 [==============================] - 290s 2s/step - loss: 0.4321 - accuracy: 0.8085 - val_loss: 0.5748 - val_accuracy: 0.7336
Epoch 6/20
137/137 [==============================] - 286s 2s/step - loss: 0.3836 - accuracy: 0.8241 - val_loss: 0.5908 - val_accuracy: 0.7345
Epoch 7/20
137/137 [==============================] - 288s 2s/step - loss: 0.3365 - accuracy: 0.8514 - val_loss: 0.6481 - val_accuracy: 0.7062